In [1]:
%%capture
from pathlib import Path

if Path.cwd().stem == "notebooks":
    %cd ..
    %load_ext autoreload
    %autoreload 2

In [ ]:
import logging
from pathlib import Path

import altair as alt
import holoviews as hv
import hvplot.polars  # noqa
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from polars import col

from src.data.database_manager import DatabaseManager
from src.features.eda import detrend_tonic_component
from src.features.scaling import scale_min_max
from src.features.utils import to_describe
from src.log_config import configure_logging
from src.plots.correlations import (
    aggregate_correlations_fisher_z,
    calculate_correlations_by_trial,
    plot_correlations_by_participant,
    plot_correlations_by_trial,
)
from src.plots.utils import prepare_multiline_hvplot

logger = logging.getLogger(__name__.rsplit(".", maxsplit=1)[-1])
configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado", "param", "numba"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [4]:
db = DatabaseManager()

In [6]:
with db:
    df = db.get_table(
        "Merged_and_Labeled_Data",
        exclude_trials_with_measurement_problems=True,
    )

# Add detrended EDA
# different paradigms: statistics and statistical learning / non-causal and causal
df = detrend_tonic_component(df)


# Add random walk column for baseline
def add_random_walk(group):
    return group.with_columns(
        random_walk=pl.Series(np.cumsum(np.random.normal(0, 1, len(group))))
    )


df = df.group_by("trial_id").map_groups(add_random_walk)

# Rename columns
df = df.with_columns(col("pupil_mean").alias("pupil"))
df

trial_id,trial_number,participant_id,timestamp,temperature,rating,eda_raw,eda_tonic,eda_phasic,ppg_raw,ppg_ibi_shimmer,heartrate,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l,pupil_mean,pupil_mean_tonic,brow_furrow,cheek_raise,mouth_open,upper_lip_raise,nose_wrinkle,normalized_timestamp,stimulus_seed,skin_patch,decreasing_intervals,major_decreasing_intervals,increasing_intervals,strictly_increasing_intervals,strictly_increasing_intervals_without_plateaus,plateau_intervals,prolonged_minima_intervals,eda_tonic_detrended,random_walk,pupil
u16,u8,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u16,u8,u16,u16,u16,u16,u16,u16,u16,f64,f64,f64
152,7,16,1.5325e6,0.0,0.04875,38.992525,39.948587,-0.956061,1341.602141,-2.294522,75.125022,3.73386,4.484003,4.468922,3.720864,4.094893,4.079086,0.00001,0.00027,0.1495,0.00254,0.00067,0.0,467,null,0,0,756,0,0,0,0,1.178289,0.774114,4.094893
152,7,16,1.5326e6,0.000109,0.096573,39.538838,39.968981,-0.430143,1382.351741,-0.309976,75.123715,3.733635,4.47222,4.475481,3.73739,4.106435,4.075112,0.00001,0.00027,0.1495,0.00254,0.00067,100.0,467,null,0,0,756,0,0,0,0,1.199474,0.767465,4.106435
152,7,16,1.5327e6,0.000479,0.138169,40.341918,39.997758,0.344159,1494.310745,-1.49892,75.122771,3.762629,4.49223,4.477659,3.752484,4.115071,4.069684,0.00001,0.000313,0.140003,0.002706,0.000823,200.0,467,null,0,0,756,0,0,0,0,1.22904,1.144021,4.115071
152,7,16,1.5328e6,0.001144,0.180811,41.012451,40.018631,0.99382,1445.770677,-1.956361,75.123707,3.79612,4.506266,4.47717,3.755816,4.116493,4.063486,0.00001,0.000359,0.122059,0.002468,0.000806,300.0,467,null,0,0,756,0,0,0,0,1.250703,0.575906,4.116493
152,7,16,1.5329e6,0.002091,0.195065,41.635092,40.042375,1.592717,1421.392597,1.112123,75.128755,3.81309,4.491845,4.478086,3.754048,4.116067,4.056209,0.00001,0.000511,0.126152,0.003189,0.001016,400.0,467,null,0,0,756,0,0,0,0,1.275237,0.634105,4.116067
152,7,16,1.5330e6,0.003334,0.228693,41.97409,40.057918,1.916172,1417.120468,-4.27699,75.135705,3.8002,4.529757,4.483909,3.754484,4.119197,4.047952,0.00001,0.00053,0.096497,0.002629,0.000899,500.0,467,null,0,0,756,0,0,0,0,1.291569,1.013051,4.119197
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
503,2,50,582830.51,0.343756,0.51625,28.258664,28.936603,-0.032404,1266.350752,-1.127491,56.805924,5.44562,5.440452,5.573725,5.740584,5.657155,5.56573,0.000071,0.000472,0.01603,0.000133,0.000441,179500.0,243,null,2515,0,0,0,0,0,0,0.264231,55.309101,5.657155
503,2,50,582930.51,0.34311,0.51625,28.241255,28.936354,-0.029987,1241.013612,-0.867424,56.529859,5.586552,5.489392,5.564452,5.731861,5.648156,5.566338,0.000108,0.00164,0.07298,0.000496,0.000563,179600.0,243,null,2515,0,0,0,0,0,0,0.264385,54.758492,5.648156


In [ ]:
# only use increasing intervals
df = df.filter(col("increasing_intervals") > 0).with_columns(
    trial_id=col("increasing_intervals")
)

In [ ]:
scale_min_max(df).hvplot(
    x="timestamp",
    y=[
        "eda_raw",
        "pupil",
        "rating",
        "temperature",
        "pupil_r_raw",
        "pupil_l_raw",
    ],
    groupby=["increasing_intervals"],
    # groupby=["participant_id", "trial_number"],
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'0dd67a00-45e3-45f7-98a9-1d18f564c1c3': {'version…

## Random Correlation Baseline

In [65]:
col1, col2 = "random_walk", "rating"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
)
plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")

alt.LayerChart(...)

In [66]:
plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr")


alt.LayerChart(...)

## Temperature / Rating

In [67]:
col1, col2 = "temperature", "rating"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr")
plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")

alt.LayerChart(...)

## Pupil / Rating

In [68]:
col1, col2 = "pupil", "rating"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr")
plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")

alt.LayerChart(...)

In [69]:
plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")


alt.LayerChart(...)

In [70]:
scale_min_max(df).hvplot(
    x="timestamp", y=["pupil", "rating", "eda_tonic"], groupby=["trial_id"]
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'b62ae9f8-588c-43ac-b83e-9f1da33380c4': {'version…

## Pupil / Temperature

In [71]:
col1, col2 = "pupil", "temperature"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
)
# plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr")
plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")

alt.LayerChart(...)

## Tonic EDA / Rating

In [72]:
col1, col2 = "eda_tonic", "rating"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr")
plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")

alt.LayerChart(...)

In [73]:
col1, col2 = "eda_tonic_detrended", "temperature"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr")
# plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")

alt.LayerChart(...)

In [74]:
scale_min_max(df).filter(col("trial_id") == 202).hvplot(
    x="timestamp", y=["pupil", "rating", "eda_tonic"]
)

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

## Phasic EDA / Rating

In [75]:
col1, col2 = "eda_phasic", "rating"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr")
plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")

alt.LayerChart(...)

## Heart Rate / Rating

In [76]:
col1, col2 = "heartrate", "rating"

corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
corr_by_participant = aggregate_correlations_fisher_z(
    corr_by_trial, f"{col1}_{col2}_corr", "participant_id", include_ci=True
)
plot_correlations_by_trial(corr_by_trial, f"{col1}_{col2}_corr")
plot_correlations_by_participant(corr_by_participant, f"{col1}_{col2}_corr")

alt.LayerChart(...)

In [77]:
scale_min_max(df).filter(col("participant_id") == 16).hvplot(
    x="timestamp", y=["heartrate", "rating"], groupby="trial_id"
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'94a8f367-9fbe-4c0d-a7bf-8a2aed542f95': {'version…

heart rate as a mediator of respiration?
-> instruct participants to exhale while pain decreases?